In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import pandas as pd
import xgboost as xgb
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.utils import class_weight
from sklearn.metrics import balanced_accuracy_score


In [5]:
#get the list of all patients' ids
def subids(file):
    file_subid=pd.read_csv(file)
    lista=[]
    count=0
    for x in range(len(file_subid['Subject ID'])):
        ids=file_subid['Subject ID'][x]
        if ids not in lista:
            lista.append(ids)
            count+=1
    return lista
lista=subids("final_final.csv")

In [6]:
#eliminate the patients who had an id that isn't in the list before
def retirar_linhas():
    lista=subids("final_final.csv")
    print(lista)
    file="todos.csv"
    file_subid=pd.read_csv(file)
    count=0

    for x in range(len(file_subid['Subject ID'])):
        ids=file_subid['Subject ID'][x]
        if ids not in lista:
            count+=1
            file_subid.drop(x,axis=0,inplace=True)
    print(count)
    #file_subid.to_csv("final.csv")
retirar_linhas()

['003_S_1057', '011_S_0241', '011_S_0362', '011_S_0861', '011_S_1080', '011_S_1282', '016_S_0769', '016_S_1121', '016_S_1326', '022_S_1394', '023_S_0042', '023_S_0126', '023_S_0217', '023_S_0388', '023_S_0604', '023_S_0625', '023_S_0887', '031_S_0830', '031_S_1066', '036_S_0869', '036_S_0976', '041_S_1010', '041_S_1425', '057_S_0839', '062_S_1299', '073_S_0518', '099_S_0054', '116_S_0649', '128_S_0227', '128_S_0608', '133_S_0638', '133_S_0727']
248


In [ ]:
#include the image ID to the correct patient ID in the final_final.csv
def incluir_imageid():
    lista=subids("568.csv")
    file="568.csv"
    file_subid=pd.read_csv(file)
    num=[]
    for i in range(2001,2569):
        num.append(i)
    images=[]
    for sub in lista:
        image=[]
        for x in range(len(file_subid['Subject'])):
            if file_subid['Subject'][x] == sub:
                image.append(file_subid['Image Data ID'][x])
        images.append(image)

    count=0
    count1=0
    file="final.csv"
    file_subid=pd.read_csv(file)
    file_subid['Image ID']=num
    for sub in lista:
        count+=1
        count1=0
        for x in range(len(file_subid['Subject ID'])):
            if file_subid['Subject ID'][x] == sub:
                count1+=1
                if count1 <= len(images[count-1]):
                    #print(images[count-1][count1-1])
                    file_subid.replace(file_subid['Image ID'][x],images[count-1][count1-1],inplace=True)

       
    file_subid.to_csv("final_final.csv")
incluir_imageid()


In [ ]:
#include the MMSE and Global CDR to the correct patient ID in the final_final.csv
def incluir_MMSE_CDR():
    lista=subids("final_final.csv")

    print(lista)
    file="final_final.csv"
    file_subid=pd.read_csv(file)
    num=[]
    num1=[]
    for i in range(40,608):
        num.append(i)

    for i in range(1000,1568):
        num1.append(i)
    mmses=[]
    globalcdrs=[]
    for sub in lista:
        mmse=[]
        globalcdr=[]
        for x in range(len(file_subid['Subject ID'])):
            if file_subid['Subject ID'][x] == sub:
                mmse.append(file_subid['MMSE Total Score'][x])
                globalcdr.append(file_subid['Global CDR'][x])
        mmses.append(mmse)
        globalcdrs.append(globalcdr)

    count=0
    count1=0
    file="568.csv"
    file_subid=pd.read_csv(file)
    file_subid['MMSE Total Score']=num
    file_subid['Global CDR']=num1
    for sub in lista:
        count+=1
        count1=0
        for x in range(len(file_subid['Subject'])):
            if file_subid['Subject'][x] == sub:
                count1+=1
                if count1 <= len(mmses[count-1]):
                    #print(images[count-1][count1-1])
                    file_subid.replace(file_subid['MMSE Total Score'][x],mmses[count-1][count1-1],inplace=True)
                    file_subid.replace(file_subid['Global CDR'][x],globalcdrs[count-1][count1-1],inplace=True)

    file_subid.to_csv("final_final1.csv")
incluir_MMSE_CDR()

In [ ]:
#see all the radiomics features
def feature_analysis(file):
    radiomics_features=pd.read_csv(file)
    #radiomics_features.isna().values.sum()
    #radiomics_features=radiomics_features.dropna()
    cols = radiomics_features.columns
    print(radiomics_features.select_dtypes(include=['category']).columns.tolist())
    print(radiomics_features.select_dtypes(exclude=['category']).columns.tolist())
    #num_cols = radiomics_features._get_numeric_data().columns
    #print(list(set(cols) - set(num_cols)))
    #columns_to_drop = ['diagnostics_Image-original_Minimum','diagnostics_Image-original_Mean','diagnostics_Image-original_Maximum']
    #radiomics_features = radiomics_features.drop(columns_to_drop, axis = 1)
    #print(num_cols)
    print(radiomics_features.shape)
    return radiomics_features

file="features.csv"
radiomics_features=feature_analysis(file)


In [ ]:
#drop the columns of all features that aren't numeric
def drop_columns(radiomics_features):
    my_type = 'float64'

    dtypes = radiomics_features.dtypes.to_dict()
    
    for col_nam, typ in dtypes.items():

         if (typ != my_type) and col_nam != "ID":
                print(col_nam)
                radiomics_features.drop(col_nam, axis = 1,inplace = True)
                
    radiomics_features.drop("diagnostics_Image-original_Mean", axis = 1,inplace = True)
    radiomics_features.drop("diagnostics_Image-original_Maximum", axis = 1,inplace = True)
    print(radiomics_features.shape)
    return radiomics_features
cleaned_features=drop_columns(radiomics_features)

In [ ]:
#get a csv file with all the numeric radiomics features
dtypes = cleaned_features.dtypes.to_dict()
for col_nam, typ in dtypes.items():
        count=0
        num = cleaned_features[col_nam][0]
        for x in range(1,len(cleaned_features[col_nam])):
            if cleaned_features[col_nam][x]!=num:
                count+=1
        
        if count==0:
            cleaned_features.drop(col_nam, axis = 1,inplace = True)
        
        
print(cleaned_features.shape)
cleaned_features.to_csv("cleaned_features.csv")

In [19]:
#drop all columns that had an image technique different than the ones mentioned in this cell
file="cleaned_features.csv"
df=pd.read_csv(file)
count=0
for i in range(len(df["ID"])):
    if "B1-calibration_Body" not in df["ID"][i] and "B1-calibration_Head" not in df["ID"][i] and "MPRAGE" not in df["ID"][i] and "MPRAGE_Repeat" not in df["ID"][i]:
        df.drop(i,inplace=True)
print(df.shape)
df.to_csv("final_cleaned_features.csv")

(556, 1825)


In [3]:
#see the patients who had exams with those techniques
file="final_final.csv"
df=pd.read_csv(file)
lista=subids("final_final.csv")
lista_ids=[]
for id in lista:
    for i in range(len(df["Subject ID"])):

        if (df["Subject ID"][i]==id) and (df["Description"][i]!="B1-calibration Body") and (df["Description"][i]!="B1-calibration Head") and (df["Description"][i]!="MPRAGE Repeat") and (df["Description"][i]!="MPRAGE"):
            if id not in lista_ids:
                lista_ids.append(id)
print(lista_ids)
            

['016_S_1121', '016_S_1326', '022_S_1394', '023_S_0625', '031_S_1066', '116_S_0649', '128_S_0227', '128_S_0608', '133_S_0638', '133_S_0727']


In [7]:
#eliminate those exams
file="final_final.csv"
df=pd.read_csv(file)
lista_pos=[]
for i in range(len(df['Subject ID'])):
    for id in lista_ids:
        if (df['Subject ID'][i]==id):
            lista_pos.append(i)


mod=df.drop(lista_pos)
mod.to_csv("no_bad_descript.csv")
    

In [22]:
#put in a csv file the final exams (386)
file="cleaned_features.csv"
df=pd.read_csv(file)
lista_pos=[]
for i in range(len(df['ID'])):
    for id in lista_ids:
        if (id in df['ID'][i]):
            lista_pos.append(i)

mod=df.drop(lista_pos)
mod.to_csv("final_cleaned_features.csv")

In [21]:
#add sex, Age and MMSE to the file before
lista=subids("no_bad_descript.csv")
descript=["B1-calibration Body","B1-calibration Head","MPRAGE","MPRAGE Repeat"]
file="no_bad_descript.csv"
df=pd.read_csv(file)
lista_age=[]
lista_sex=[]
lista_MMSE=[]
for id in lista:
    for desc in descript:
    
        for i in range(len(df["Subject ID"])):
            if df["Subject ID"][i]==id and df["Description"][i] == desc:
                lista_age.append(df["Age"][i])
                if df["Sex"][i]=="F":
                    lista_sex.append(0)
                else:
                     lista_sex.append(1)
                lista_MMSE.append(df["MMSE Total Score"][i])

file="final_cleaned_features.csv"
df=pd.read_csv(file)
df["Sex"]=lista_sex
df["Age"]=lista_age
df["MMSE"]=lista_MMSE

df.to_csv("final_cleaned_features.csv")


In [29]:
#classify the exams accordingly to the MMSE Score
descript=["B1-calibration_Body","B1-calibration_Head","MPRAGE_br","MPRAGE_Repeat"]


def prepare_exams2(file):
    
    final = pd.read_csv(file)
    
    lista=subids("final_final.csv")
    
    
    
    listatotal_com_exames=[]
    for id in lista:
        
        for desc in descript:
            lista_com_exames=[]
            for x in range(len(final["ID"])):
                
                if id in final["ID"][x] and (desc in final["ID"][x]):
                    
                    if final["MMSE"][x]> 26 and final["MMSE"][x] <=30:
                        lista_com_exames.append(6)
                   # if final["MMSE"][x]>= 22 and final["MMSE"][x] <26:
                       # lista_com_exames.append(7)       
                    if final["MMSE"][x]> 21 and final["MMSE"][x] <=26:
                        lista_com_exames.append(7)   
                    if final["MMSE"][x]> 10 and final["MMSE"][x] <=21:
                        lista_com_exames.append(8) 
            listatotal_com_exames.append(lista_com_exames)
      
    lista_final=[]
    for i in listatotal_com_exames:
        if i!=[]:
            lista_final.append(i)
   
        
                
    return lista_final

lista=prepare_exams2('final_cleaned_features.csv')    
print(lista)


#count=0
#for i in range(len(lista)):
#    if lista[i]==8:
#        count+=1
#print(count)

[[7, 6, 7, 7], [7, 6, 7, 7], [7, 6, 7, 7], [7, 6, 7, 7], [7, 7, 6, 6], [7, 7, 6, 6], [7, 7, 6, 6], [7, 7, 6, 6], [7, 7, 7, 8], [7, 7, 7, 8], [7, 7, 7, 7, 8], [7, 7, 7, 8], [6, 7, 8, 8], [6, 7, 8, 8], [6, 7, 8, 8], [6, 7, 8, 8], [7, 6, 7, 7], [7, 6, 7, 7], [7, 6, 7, 7], [7, 6, 7, 7], [6, 7, 7, 8], [6, 7, 7, 8], [6, 7, 7, 8], [6, 6, 7, 7, 8], [6, 6, 6, 6, 7], [6, 6, 6, 6, 7], [6, 6, 6, 6, 7], [6, 6, 6, 6, 7], [6, 7, 7, 7], [6, 6, 7, 7, 7], [6, 7, 7, 7], [6, 7, 7, 7], [7, 7, 7, 7], [7, 7, 7, 7], [7, 7, 7, 7], [7, 7, 7, 7], [7, 6, 7, 7], [7, 6, 7, 7], [7, 6, 7, 7], [7, 6, 7, 7], [7, 7, 6, 6, 7, 7, 8, 8], [7, 7, 6, 6, 7, 7, 8, 8], [7, 7, 6, 6, 7, 7, 8, 8], [7, 7, 6, 6, 7, 7, 8, 8], [7, 7, 7, 7, 7, 8, 8, 7, 7], [7, 7, 7, 7, 8, 8, 7, 7], [7, 7, 7, 7, 8, 8, 7, 7, 7], [7, 7, 7, 7, 8, 8, 7, 7], [8, 7, 8, 8, 8], [8, 7, 8, 8], [8, 7, 8, 8], [8, 7, 8, 8], [6, 7, 6], [6, 7, 6], [6, 7, 6], [6, 7, 6], [7, 7, 7, 8], [7, 7, 7, 8], [7, 7, 7, 8], [7, 7, 7, 8], [7, 7, 7, 7], [7, 7, 7, 7], [7, 7, 7, 7], [7,

In [32]:
#Perform the transitions
def realizar_transicoes():
    lista=prepare_exams2('final_cleaned_features.csv')
    
    listatotal_final=[]
    for sublista in lista:
        #print(listatotal_final)
        lista_final=[]
        #print(sublista)
        num = sublista[0]
        lista_final.append(num)
        for i in range(1,len(sublista)):

            if sublista[i] == num:
                lista_final.append(sublista[i])

            else:
                if sublista[i] > num:
                    if sublista[i]==7 and sublista[i-1]==6:
                        num=sublista[i]
                        for j in range(len(lista_final)):
                            if  lista_final[j]==6:
                                lista_final[j]=0
                        lista_final.append(sublista[i])

                    if  sublista[i]==8 and sublista[i-1]==6:
                        num=sublista[i]
                        for j in range(len(lista_final)):
                            if  lista_final[j]==6:
                                 lista_final[j]=1

                        lista_final.append(sublista[i])

                    if  sublista[i]==8 and sublista[i-1]==7:
                        num=sublista[i]
                        for j in range(len(lista_final)):
                            if lista_final[j]==7:
                                lista_final[j]=2

                        lista_final.append(sublista[i])

                else:
                    num = sublista[i]
                    lista_final.append(num)

        listatotal_final.append(lista_final)
    return listatotal_final
lista=realizar_transicoes()
count=0
for i in lista:
    for j in i:
        if j ==0:
            count+=1
print(count)
print(lista)
lista_final=[]
for i in lista:
    for j in i:
        lista_final.append(j)
file="final_cleaned_features.csv"
df=pd.read_csv(file)
df["Transitions"]=lista_final
#df.to_csv("final_cleaned_features.csv")

66
[[7, 0, 7, 7], [7, 0, 7, 7], [7, 0, 7, 7], [7, 0, 7, 7], [7, 7, 6, 6], [7, 7, 6, 6], [7, 7, 6, 6], [7, 7, 6, 6], [2, 2, 2, 8], [2, 2, 2, 8], [2, 2, 2, 2, 8], [2, 2, 2, 8], [0, 2, 8, 8], [0, 2, 8, 8], [0, 2, 8, 8], [0, 2, 8, 8], [7, 0, 7, 7], [7, 0, 7, 7], [7, 0, 7, 7], [7, 0, 7, 7], [0, 2, 2, 8], [0, 2, 2, 8], [0, 2, 2, 8], [0, 0, 2, 2, 8], [0, 0, 0, 0, 7], [0, 0, 0, 0, 7], [0, 0, 0, 0, 7], [0, 0, 0, 0, 7], [0, 7, 7, 7], [0, 0, 7, 7, 7], [0, 7, 7, 7], [0, 7, 7, 7], [7, 7, 7, 7], [7, 7, 7, 7], [7, 7, 7, 7], [7, 7, 7, 7], [7, 0, 7, 7], [7, 0, 7, 7], [7, 0, 7, 7], [7, 0, 7, 7], [2, 2, 0, 0, 2, 2, 8, 8], [2, 2, 0, 0, 2, 2, 8, 8], [2, 2, 0, 0, 2, 2, 8, 8], [2, 2, 0, 0, 2, 2, 8, 8], [2, 2, 2, 2, 2, 8, 8, 7, 7], [2, 2, 2, 2, 8, 8, 7, 7], [2, 2, 2, 2, 8, 8, 7, 7, 7], [2, 2, 2, 2, 8, 8, 7, 7], [8, 2, 8, 8, 8], [8, 2, 8, 8], [8, 2, 8, 8], [8, 2, 8, 8], [0, 7, 6], [0, 7, 6], [0, 7, 6], [0, 7, 6], [2, 2, 2, 8], [2, 2, 2, 8], [2, 2, 2, 8], [2, 2, 2, 8], [7, 7, 7, 7], [7, 7, 7, 7], [7, 7, 7, 7], 

In [115]:
#get all the ages
descript=["B1-calibration_Body","B1-calibration_Head","MPRAGE_br","MPRAGE_Repeat"]


def prepare_age(file):
    
    final = pd.read_csv(file)
    
    lista=subids("final_final.csv")
    
    
    
    listatotal_com_idades=[]
    for id in lista:
        
        for desc in descript:
            lista_com_idades=[]
            for x in range(len(final["ID"])):
                
                if id in final["ID"][x] and (desc in final["ID"][x]):
                    
                    
                    lista_com_idades.append(final["Age"][x])
                  
            listatotal_com_idades.append(lista_com_idades)
      
    lista_final=[]
    for i in listatotal_com_idades:
        if i!=[]:
            lista_final.append(i)
   
        
                
    return lista_final

lista=prepare_age('final_cleaned_features.csv')    
print(lista)

    

[[62.3, 62.6, 63.7, 64.5], [62.3, 62.6, 63.7, 64.5], [62.3, 62.6, 63.7, 64.5], [62.3, 62.6, 63.7, 64.5], [82.4, 82.9, 84.0, 84.9], [82.4, 82.9, 84.0, 84.9], [82.4, 82.9, 84.0, 84.9], [82.4, 82.9, 84.0, 84.9], [71.1, 71.6, 72.6, 73.6], [71.1, 71.6, 72.6, 73.6], [71.1, 71.6, 71.6, 72.6, 73.6], [71.1, 71.6, 72.6, 73.6], [87.8, 88.3, 89.3, 90.3], [87.8, 88.3, 89.3, 90.3], [87.8, 88.3, 89.3, 90.3], [87.8, 88.3, 89.3, 90.3], [82.0, 82.6, 83.6, 84.6], [82.0, 82.6, 83.6, 84.6], [82.0, 82.6, 83.6, 84.6], [82.0, 82.6, 83.6, 84.6], [77.7, 78.2, 79.2, 80.2], [77.7, 78.2, 79.2, 80.2], [77.7, 78.2, 79.2, 80.2], [77.7, 77.7, 78.2, 79.2, 80.2], [62.9, 62.0, 63.3, 63.3, 65.3], [62.9, 62.0, 63.3, 63.3, 65.3], [62.9, 62.0, 63.3, 63.3, 65.3], [62.9, 62.0, 63.3, 63.3, 65.3], [73.4, 73.9, 75.0, 76.0], [73.4, 73.4, 73.9, 75.0, 76.0], [73.4, 73.9, 75.0, 76.0], [73.4, 73.9, 75.0, 76.0], [78.2, 78.9, 79.8, 80.8], [78.2, 78.9, 79.8, 80.8], [78.2, 78.9, 79.8, 80.8], [78.2, 78.9, 79.8, 80.8], [84.2, 84.7, 85.7, 86

In [116]:
#perform the age difference
def realizar_dif_age():
    lista_exams=prepare_exams2('final_cleaned_features.csv')
    lista_age=prepare_age('final_cleaned_features.csv')  
    listatotal_final=[]
    listatotal_final2=[]
    for j in range(len(lista_exams)):
        #print(listatotal_final)
        lista_final=[]
        lista_final2=[]
        #print(sublista)
        num = lista_exams[j][0]
        lista_final.append(num)
        age=lista_age[j][0]
        lista_final2.append(age)
        for i in range(1,len(lista_exams[j])):

            if lista_exams[j][i] == num:
                lista_final.append(lista_exams[j][i])
                lista_final2.append(lista_age[j][i])

            else:
                if lista_exams[j][i] > num:
                    if lista_exams[j][i]==7 and lista_exams[j][i-1]==6:
                        num=lista_exams[j][i]
                        age=lista_age[j][i]
                        for k in range(len(lista_final)):
                            if  lista_final[k]==6:
                                lista_final[k]=0
                                lista_final2[k]=age-lista_final2[k]
                                
                        lista_final.append(lista_exams[j][i])
                        lista_final2.append(lista_age[j][i])

                    if  lista_exams[j][i]==8 and lista_exams[j][i-1]==6:
                        num=lista_exams[j][i]
                        age=lista_age[j][i]
                        for k in range(len(lista_final)):
                            if  lista_final[k]==6:
                                lista_final[k]=1
                                lista_final2[k]=age-lista_final2[k]
                                
                        lista_final.append(lista_exams[j][i])
                        lista_final2.append(lista_age[j][i])
                        

                    if  lista_exams[j][i]==8 and lista_exams[j][i-1]==7:
                        num=lista_exams[j][i]
                        age=lista_age[j][i]
                        for k in range(len(lista_final)):
                            if lista_final[k]==7:
                                lista_final[k]=2
                                lista_final2[k]=age-lista_final2[k]

                        lista_final.append(lista_exams[j][i])
                        lista_final2.append(lista_age[j][i])

                else:
                   
                    num = lista_exams[j][i]
                    age=lista_age[j][i]
                    lista_final.append(num)
                    lista_final2.append(age)

        listatotal_final2.append(lista_final2)
    return listatotal_final2
lista=realizar_dif_age()
lista_final_age=[]
count=0
for i in lista:
    lista_age=[]
    for j in i:
        if j >30:
            j=-1
            
            lista_age.append(j)
        else:
            count+=1
            lista_age.append(j)
    lista_final_age.append(lista_age)
print(count)
print(lista_final_age)
lista_final=[]
for i in lista_final_age:
    for j in i:
        lista_final.append(j)
file="final_cleaned_features.csv"
df=pd.read_csv(file)
df["DifAge"]=lista_final
df.to_csv("final_cleaned_features.csv")

168
[[-1, 1.1000000000000014, -1, -1], [-1, 1.1000000000000014, -1, -1], [-1, 1.1000000000000014, -1, -1], [-1, 1.1000000000000014, -1, -1], [-1, -1, -1, -1], [-1, -1, -1, -1], [-1, -1, -1, -1], [-1, -1, -1, -1], [2.5, 2.0, 1.0, -1], [2.5, 2.0, 1.0, -1], [2.5, 2.0, 2.0, 1.0, -1], [2.5, 2.0, 1.0, -1], [0.5, 1.0, -1, -1], [0.5, 1.0, -1, -1], [0.5, 1.0, -1, -1], [0.5, 1.0, -1, -1], [-1, 1.0, -1, -1], [-1, 1.0, -1, -1], [-1, 1.0, -1, -1], [-1, 1.0, -1, -1], [0.5, 2.0, 1.0, -1], [0.5, 2.0, 1.0, -1], [0.5, 2.0, 1.0, -1], [0.5, 0.5, 2.0, 1.0, -1], [2.3999999999999986, 3.299999999999997, 2.0, 2.0, -1], [2.3999999999999986, 3.299999999999997, 2.0, 2.0, -1], [2.3999999999999986, 3.299999999999997, 2.0, 2.0, -1], [2.3999999999999986, 3.299999999999997, 2.0, 2.0, -1], [0.5, -1, -1, -1], [0.5, 0.5, -1, -1, -1], [0.5, -1, -1, -1], [0.5, -1, -1, -1], [-1, -1, -1, -1], [-1, -1, -1, -1], [-1, -1, -1, -1], [-1, -1, -1, -1], [-1, 1.0, -1, -1], [-1, 1.0, -1, -1], [-1, 1.0, -1, -1], [-1, 1.0, -1, -1], [2.3

In [117]:
#get all MMSE score
descript=["B1-calibration_Body","B1-calibration_Head","MPRAGE_br","MPRAGE_Repeat"]


def prepare_MMSE(file):
    
    final = pd.read_csv(file)
    
    lista=subids("final_final.csv")
    
    
    
    listatotal_com_idades=[]
    for id in lista:
        
        for desc in descript:
            lista_com_idades=[]
            for x in range(len(final["ID"])):
                
                if id in final["ID"][x] and (desc in final["ID"][x]):
                    
                    
                    lista_com_idades.append(final["MMSE"][x])
                  
            listatotal_com_idades.append(lista_com_idades)
      
    lista_final=[]
    for i in listatotal_com_idades:
        if i!=[]:
            lista_final.append(i)
   
        
                
    return lista_final

lista=prepare_MMSE('final_cleaned_features.csv')    
print(lista)

[[25.0, 28.0, 23.0, 23.0], [25.0, 28.0, 23.0, 23.0], [25.0, 28.0, 23.0, 23.0], [25.0, 28.0, 23.0, 23.0], [25.0, 25.0, 27.0, 29.0], [25.0, 25.0, 27.0, 29.0], [25.0, 25.0, 27.0, 29.0], [25.0, 25.0, 27.0, 29.0], [25.0, 24.0, 24.0, 20.0], [25.0, 24.0, 24.0, 20.0], [25.0, 24.0, 24.0, 24.0, 20.0], [25.0, 24.0, 24.0, 20.0], [27.0, 26.0, 20.0, 16.0], [27.0, 26.0, 20.0, 16.0], [27.0, 26.0, 20.0, 16.0], [27.0, 26.0, 20.0, 16.0], [24.0, 28.0, 23.0, 24.0], [24.0, 28.0, 23.0, 24.0], [24.0, 28.0, 23.0, 24.0], [24.0, 28.0, 23.0, 24.0], [28.0, 24.0, 24.0, 16.0], [28.0, 24.0, 24.0, 16.0], [28.0, 24.0, 24.0, 16.0], [28.0, 28.0, 24.0, 24.0, 16.0], [27.0, 27.0, 28.0, 28.0, 22.0], [27.0, 27.0, 28.0, 28.0, 22.0], [27.0, 27.0, 28.0, 28.0, 22.0], [27.0, 27.0, 28.0, 28.0, 22.0], [28.0, 24.0, 22.0, 23.0], [28.0, 28.0, 24.0, 22.0, 23.0], [28.0, 24.0, 22.0, 23.0], [28.0, 24.0, 22.0, 23.0], [24.0, 26.0, 26.0, 22.0], [24.0, 26.0, 26.0, 22.0], [24.0, 26.0, 26.0, 22.0], [24.0, 26.0, 26.0, 22.0], [26.0, 27.0, 26.0, 23

In [118]:
#perform the MMSE difference
def realizar_dif_MMSE():
    lista_exams=prepare_exams2('final_cleaned_features.csv')
    lista_age=prepare_MMSE('final_cleaned_features.csv')  
    listatotal_final=[]
    listatotal_final2=[]
    for j in range(len(lista_exams)):
        #print(listatotal_final)
        lista_final=[]
        lista_final2=[]
        #print(sublista)
        num = lista_exams[j][0]
        lista_final.append(num)
        age=lista_age[j][0]
        lista_final2.append(age)
        for i in range(1,len(lista_exams[j])):

            if lista_exams[j][i] == num:
                lista_final.append(lista_exams[j][i])
                lista_final2.append(lista_age[j][i])

            else:
                if lista_exams[j][i] > num:
                    if lista_exams[j][i]==7 and lista_exams[j][i-1]==6:
                        num=lista_exams[j][i]
                        age=lista_age[j][i]
                        for k in range(len(lista_final)):
                            if  lista_final[k]==6:
                                lista_final[k]=0
                                lista_final2[k]=age-lista_final2[k]
                                
                        lista_final.append(lista_exams[j][i])
                        lista_final2.append(lista_age[j][i])

                    if  lista_exams[j][i]==8 and lista_exams[j][i-1]==6:
                        num=lista_exams[j][i]
                        age=lista_age[j][i]
                        for k in range(len(lista_final)):
                            if  lista_final[k]==6:
                                lista_final[k]=1
                                lista_final2[k]=age-lista_final2[k]
                                
                        lista_final.append(lista_exams[j][i])
                        lista_final2.append(lista_age[j][i])
                        

                    if  lista_exams[j][i]==8 and lista_exams[j][i-1]==7:
                        num=lista_exams[j][i]
                        age=lista_age[j][i]
                        for k in range(len(lista_final)):
                            if lista_final[k]==7:
                                lista_final[k]=2
                                lista_final2[k]=age-lista_final2[k]

                        lista_final.append(lista_exams[j][i])
                        lista_final2.append(lista_age[j][i])

                else:
                    
                    num = lista_exams[j][i]
                    age=lista_age[j][i]
                    lista_final.append(num)
                    lista_final2.append(age)

        listatotal_final2.append(lista_final2)
    return listatotal_final2
lista=realizar_dif_MMSE()
lista_final_age=[]
count=0
for i in lista:
    lista_age=[]
    for j in i:
        if j >0:
            j=0
            
            lista_age.append(j)
        else:
            count+=1
            lista_age.append(j)
    lista_final_age.append(lista_age)
print(count)
print(lista_final_age)
lista_final=[]
for i in lista_final_age:
    for j in i:
        lista_final.append(j)
file="final_cleaned_features.csv"
df=pd.read_csv(file)
df["DifMMSE"]=lista_final
df.to_csv("final_cleaned_features.csv")

168
[[0, -5.0, 0, 0], [0, -5.0, 0, 0], [0, -5.0, 0, 0], [0, -5.0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [-5.0, -4.0, -4.0, 0], [-5.0, -4.0, -4.0, 0], [-5.0, -4.0, -4.0, -4.0, 0], [-5.0, -4.0, -4.0, 0], [-1.0, -6.0, 0, 0], [-1.0, -6.0, 0, 0], [-1.0, -6.0, 0, 0], [-1.0, -6.0, 0, 0], [0, -5.0, 0, 0], [0, -5.0, 0, 0], [0, -5.0, 0, 0], [0, -5.0, 0, 0], [-4.0, -8.0, -8.0, 0], [-4.0, -8.0, -8.0, 0], [-4.0, -8.0, -8.0, 0], [-4.0, -4.0, -8.0, -8.0, 0], [-5.0, -5.0, -6.0, -6.0, 0], [-5.0, -5.0, -6.0, -6.0, 0], [-5.0, -5.0, -6.0, -6.0, 0], [-5.0, -5.0, -6.0, -6.0, 0], [-4.0, 0, 0, 0], [-4.0, -4.0, 0, 0, 0], [-4.0, 0, 0, 0], [-4.0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, -1.0, 0, 0], [0, -1.0, 0, 0], [0, -1.0, 0, 0], [0, -1.0, 0, 0], [-3.0, -3.0, -1.0, -1.0, -6.0, -6.0, 0, 0], [-3.0, -3.0, -1.0, -1.0, -6.0, -6.0, 0, 0], [-3.0, -3.0, -1.0, -1.0, -6.0, -6.0, 0, 0], [-3.0, -3.0, -1.0, -1.0, -6.0, -6.0, 0, 0], [-3.0, -3.0, -3.0, -6.0, -6.0, 0, 0, 

In [28]:
#get all the exams that had transitions to a csv file
df = pd.read_csv('final_cleaned_features.csv')
dtypes = df.dtypes.to_dict()
for i in range(len(df["DifAge"])):
    if df["DifAge"][i]==-1:
        df.drop(i,inplace=True)
for col_nam, typ in dtypes.items():
    if "Unnamed" in col_nam:
        df.drop(col_nam, axis = 1,inplace = True)
print(df.shape)
df.to_csv("model_dataset.csv")

(168, 1830)


In [4]:
#normalize the data
columns=["ID","Sex","Age","MMSE","Transitions","DifAge","DifMMSE","Unnamed: 0"]
def normalize_radiomic():
    df = pd.read_csv('model_dataset.csv')
    
    data = df.drop(columns=columns)
    
    data_normalized = data.apply(lambda x:( (x - x.min()) / (x.max()-x.min())))
    
    #data_normalized.to_csv('Normalized_Features.csv', index=False)
    
    return data_normalized
   
df=normalize_radiomic()
csv = pd.read_csv('model_dataset.csv')



listaID=csv["ID"]
listaSex=csv["Sex"]
listaAge=csv["Age"]
listaMMSE=csv["MMSE"]
listaTransitions=csv["Transitions"]
listaDifAge=csv["DifAge"]
listaDifMMSE=csv["DifMMSE"]


        

        
df["ID"]=listaID
df["Sex"]=listaSex
df["Age"]=listaAge
df["DifAge"]=listaDifAge
df["MMSE"]=listaMMSE
df["DifMMSE"]=listaDifMMSE
df["Label"]=listaTransitions

df.to_csv("last_model_dataset.csv")

In [3]:
#get the exams that doesn't have transitions
file="all_transitions_no_transitions.csv"
df=pd.read_csv(file)
lista=[]
lista_difage=[]
for x in range(len(df["Transitions"])):
    if df["Transitions"][x]<3:
        df.drop(x,inplace=True)

df.to_csv("no_transitions.csv")
        

In [8]:
#normalize the data
columns=["ID","Sex","Age","MMSE","Transitions"]
def normalize_radiomic():
    df = pd.read_csv('no_transitions.csv')
    
    data = df.drop(columns=columns)
    
    data_normalized = data.apply(lambda x:( (x - x.min()) / (x.max()-x.min())))
    
    #data_normalized.to_csv('Normalized_Features.csv', index=False)
    
    return data_normalized
   
df=normalize_radiomic()
csv = pd.read_csv('no_transitions.csv')



listaID=csv["ID"]
listaSex=csv["Sex"]
listaAge=csv["Age"]
listaMMSE=csv["MMSE"]
listaTransitions=csv["Transitions"]
listaDifAge=csv["DifAge"]
listaDifMMSE=csv["DifMMSE"]


        

        
df["ID"]=listaID
df["Sex"]=listaSex
df["Age"]=listaAge
df["DifAge"]=listaDifAge
df["MMSE"]=listaMMSE
df["DifMMSE"]=listaDifMMSE
df["Label"]=listaTransitions

df.to_csv("no_transitions_normalize.csv")

In [3]:
#How many exams with CN, MCI, AD (sex and age)
df = pd.read_csv('final_cleaned_features.csv')
count_male_cn=0
count_female_cn=0
count_male_mci=0
count_female_mci=0
count_male_ad=0
count_female_ad=0
minimo_age_cn=1000
maximo_age_cn=0
minimo_age_mci=1000
maximo_age_mci=0
minimo_age_ad=1000
maximo_age_ad=0
count_cn=0
count_mci=0
count_ad=0

for x in range(len(df["ID"])):
    if df["MMSE"][x]> 26 and df["MMSE"][x] <=30:
        count_cn+=1
        if df["Age"][x] >maximo_age_cn:
            maximo_age_cn=df["Age"][x]
        if df["Age"][x] < minimo_age_cn:
            minimo_age_cn=df["Age"][x]
        if df["Sex"][x]==0:
            count_female_cn+=1
        else:
            count_male_cn+=1
            
    if df["MMSE"][x]> 21 and df["MMSE"][x] <=26:
        count_mci+=1
        if df["Age"][x] >maximo_age_mci:
            maximo_age_mci=df["Age"][x]
        if df["Age"][x] < minimo_age_mci:
            minimo_age_mci=df["Age"][x]
        if df["Sex"][x]==0:
            count_female_mci+=1
        else:
            count_male_mci+=1   
        
    if df["MMSE"][x]> 10 and df["MMSE"][x] <=21:
        count_ad+=1
        if df["Age"][x] >maximo_age_ad:
            maximo_age_ad=df["Age"][x]
        if df["Age"][x] < minimo_age_ad:
            minimo_age_ad=df["Age"][x]
        if df["Sex"][x]==0:
            count_female_ad+=1
        else:
            count_male_ad+=1

print("CN:", count_cn)
print("MCI:", count_mci)
print("AD:", count_ad)            
print("Male_CN:", count_male_cn)
print("Female_CN:", count_female_cn)
print("Male_MCI:",count_male_mci)
print("Female_MCI:",count_female_mci)
print("Male_AD:", count_male_ad)
print("Female_AD:", count_female_ad)
print("Minimo_idade_CN:", minimo_age_cn)
print("Maximo_idade_CN:", maximo_age_cn)
print("Minimo_idade_MCI:", minimo_age_mci)
print("Maximo_idade_MCI:", maximo_age_mci)
print("Minimo_idade_AD:", minimo_age_ad)
print("Maximo_idade_AD:", maximo_age_ad)

CN: 90
MCI: 223
AD: 73
Male_CN: 73
Female_CN: 17
Male_MCI: 114
Female_MCI: 109
Male_AD: 40
Female_AD: 33
Minimo_idade_CN: 62.0
Maximo_idade_CN: 87.8
Minimo_idade_MCI: 62.3
Maximo_idade_MCI: 89.7
Minimo_idade_AD: 73.6
Maximo_idade_AD: 90.3
